# HashubDocApp SDK — Hızlı Başlangıç ve Test Defteri
**Tarih:** 2025-08-15

Bu notebook, **HashubDocApp Python SDK**'sını hızlıca test etmek için hazırlandı. Bölümler:
1. Kurulum ve Ortam
2. Hızlı Başlangıç (Quick Start)
3. Çekirdek Metodlar (convert_fast, convert_smart, convert_doc, convert_html_string)
4. Toplu İşleme (batch_*)
5. İş Yönetimi (status/wait/result/cancel)
6. Dil Yardımcısı (LanguageHelper)
7. Görüntü İyileştirme (Enhancement Presets)
8. Hata Yönetimi ve İpuçları

> **Not:** Yerel dosya yollarını kendi ortamınıza göre düzenleyin. Bu defter Colab'da da çalışır.


## 1) Kurulum ve Ortam

In [ ]:
# Eğer ortamınızda kurulu değilse aşağıyı açın (Colab veya lokal).
# !pip install -U pip
# !pip install hashub-docapp
# İsteğe bağlı: progress bar'lar ve tablo görüntüleme için ek paketler
# !pip install rich tqdm pandas


## 2) API Anahtarı ve İstemci (Client)

In [ ]:
import os
from hashub_docapp import DocAppClient

# >>> AYAR GEREKİR <<<
# En güvenlisi: ortam değişkeni ile kullanın
# os.environ['HASHUB_API_KEY'] = 'your_api_key_here'

API_KEY = os.getenv('HASHUB_API_KEY', 'your_api_key_here')  # Colab için fallback
BASE_URL = os.getenv('HASHUB_BASE_URL', 'https://doc.hashub.dev/api/v1')

client = DocAppClient(
    api_key=API_KEY,
    base_url=BASE_URL,
    timeout=(30, 120),
    max_retries=3,
    rate_limit_delay=2.0
)
print('Client hazır. Base URL:', BASE_URL)


## 3) Hızlı Başlangıç (Quick Start)

In [ ]:
# Örnek dosya yollarını kendinize göre değiştirin
PDF_PATH = 'sample_docs/invoice_tr.pdf'   # Örnek: Türkçe bir PDF
IMG_PATH = 'sample_docs/photo_scan.jpg'   # Örnek: Fotoğraf ile tarama

# FAST OCR — sade ve hızlı metin çıkarımı
try:
    text_fast = client.convert_fast(PDF_PATH, language='tr', output='txt')
    print('--- FAST OCR (TXT) ---\n', text_fast[:500])
except Exception as e:
    print('FAST OCR hata:', e)

# SMART OCR — karmaşık düzeni korur, markdown/json da alabilirsiniz
try:
    md_smart = client.convert_smart(PDF_PATH, output='markdown')
    print('--- SMART OCR (MD) Özet ---\n', md_smart[:500])
except Exception as e:
    print('SMART OCR hata:', e)


## 4) Çekirdek Metodlar

In [ ]:
# 4.1 convert_fast — dil ve enhancement ile
try:
    text_enhanced = client.convert_fast(IMG_PATH, language='tr', enhancement='scan_low_dpi', output='txt')
    print('Enhancement ile FAST OCR çıktı (ilk 400 karakter):\n', text_enhanced[:400])
except Exception as e:
    print('convert_fast hata:', e)

# 4.2 convert_smart — markdown/json ile düzen/tabloları koru
try:
    smart_json = client.convert_smart(PDF_PATH, output='json')
    # Büyük olabilir; sadece tip ve bazı anahtarları gösterelim
    print('SMART JSON tip:', type(smart_json))
    print('SMART JSON uzunluk/özeti:', str(smart_json)[:400])
except Exception as e:
    print('convert_smart hata:', e)

# 4.3 convert_doc — Office (docx/xlsx/pptx) -> markdown/txt
try:
    DOCX_PATH = 'sample_docs/sample_contract.docx'
    md_from_doc = client.convert_doc(DOCX_PATH, output='markdown')
    print('DOCX -> MD (ilk 400):\n', md_from_doc[:400])
except Exception as e:
    print('convert_doc hata:', e)

# 4.4 convert_html_string — ham HTML -> markdown/txt
try:
    html = '<h1>Başlık</h1><p>İçerik paragrafı <b>kalın</b> metin.</p>'
    md_from_html = client.convert_html_string(html, output='markdown')
    print('HTML -> MD:\n', md_from_html)
except Exception as e:
    print('convert_html_string hata:', e)


## 5) Toplu İşleme (Batch)

In [ ]:
from pathlib import Path

INPUT_DIR = Path('sample_docs')
OUTPUT_DIR = Path('output_docs')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 5.1 batch_convert_auto — dosya tipine göre en uygun modu seçer
try:
    res_auto = client.batch_convert_auto(
        directory=INPUT_DIR,
        save_to=OUTPUT_DIR,
        language='tr',
        output_format='markdown',
        recursive=True,
        show_progress=True,
        max_workers=4,
        timeout=900
    )
    print('AUTO batch özet:', {k: res_auto.get(k) for k in ['processed_count','success_count','failed_count']})
except Exception as e:
    print('batch_convert_auto hata:', e)

# 5.2 batch_convert_fast — PDF/IMG için hızlı
try:
    res_fast = client.batch_convert_fast(
        directory=INPUT_DIR,
        save_to=OUTPUT_DIR,
        language='tr',
        enhancement=None,
        output_format='txt',
        recursive=True,
        show_progress=True,
        max_workers=5,
        timeout=300
    )
    print('FAST batch özet:', {k: res_fast.get(k) for k in ['processed_count','success_count','failed_count']})
except Exception as e:
    print('batch_convert_fast hata:', e)

# 5.3 batch_convert_smart — karmaşık düzenli belgeler için
try:
    res_smart = client.batch_convert_smart(
        directory=INPUT_DIR,
        save_to=OUTPUT_DIR,
        output_format='markdown',
        recursive=True,
        show_progress=True,
        max_workers=3,
        timeout=600
    )
    print('SMART batch özet:', {k: res_smart.get(k) for k in ['processed_count','success_count','failed_count']})
except Exception as e:
    print('batch_convert_smart hata:', e)


## 6) İş Yönetimi (Durum, Bekleme, Sonuç, İptal)

In [ ]:
# convert_* metodları çoğunlukla senkron içerik döndürür.
# Eğer async/queue bazlı bir 'job_id' dönüyorsa (örn. büyük dosyalar) aşağıdaki akış örnektir.

job_id = None  # Örnek. Gerçek id'yi API dönüşünden alın.

if job_id:
    try:
        status = client.get_status(job_id)
        print('Durum:', status)
    except Exception as e:
        print('get_status hata:', e)

    try:
        final = client.wait(job_id, interval=2.0, timeout=300)
        print('Final durum:', final)
    except Exception as e:
        print('wait hata:', e)

    try:
        result = client.get_result(job_id)
        print('Sonuç anahtarları:', list(result.keys()))
    except Exception as e:
        print('get_result hata:', e)

    # İptal örneği
    # try:
    #     client.cancel(job_id)
    # except Exception as e:
    #     print('cancel hata:', e)
else:
    print('Bu örnekte job_id yok. Büyük dosyalarda/queue modlarında kullanın.')


## 7) Dil Yardımcısı (LanguageHelper)

In [ ]:
from hashub_docapp.languages import LanguageHelper

try:
    langs = LanguageHelper.list_languages()
    print('Desteklenen dil sayısı:', len(langs))
    # İlk 10 dil örnek gösterimi
    for item in langs[:10]:
        print(item)
    print('Türkçe info:', LanguageHelper.get_language_info('tr'))
except Exception as e:
    print('LanguageHelper hata:', e)


## 8) Görüntü İyileştirme (Enhancement Presets)

In [ ]:
enhancements = [
    'document_crisp','scan_low_dpi','camera_shadow','photocopy_faded','inverted_scan',
    'noisy_dots','tables_fine','receipt_thermal','newspaper_moire','fax_low_quality','blueprint'
]
print('Örnek preset listesi:', enhancements)

# Hızlı kıyas için aynı görüntüde farklı preset denemeleri (zaman ve kota maliyetine dikkat!)
TEST_IMG = IMG_PATH  # kendi örneğinizle değiştirin
for preset in enhancements[:3]:  # ilk 3 ile kısa demo
    try:
        out = client.convert_fast(TEST_IMG, language='tr', enhancement=preset, output='txt')
        print(f'Preset={preset} | Çıktı (ilk 250):\n', out[:250])
    except Exception as e:
        print(f'Preset={preset} hata:', e)


## 9) Hata Yönetimi (Exceptions)

In [ ]:
from hashub_docapp.exceptions import (
    AuthenticationError, RateLimitError, ProcessingError, ValidationError
)

try:
    # Bilinçli bir hata tetiklemek için geçersiz yol verelim (örnek)
    _ = client.convert_fast('non_existent.pdf')
except AuthenticationError:
    print('❌ API anahtarı geçersiz')
except RateLimitError:
    print('⏳ Rate limit aşıldı, bekleyip tekrar deneyin')
except ProcessingError as e:
    print('💥 İşleme hatası:', e)
except ValidationError as e:
    print('📝 Doğrulama hatası:', e)
except FileNotFoundError:
    print('📁 Dosya bulunamadı (beklenen uyarı).')
except Exception as e:
    print('Beklenmeyen hata:', e)


## 10) Performans İpuçları ve Sorun Giderme


**Performans İpuçları**
- Basit belgeler için `convert_fast`, karmaşık düzen için `convert_smart` tercih edin.
- Toplu işlemlerde `max_workers` değerini kota limitlerine göre ayarlayın.
- Dili doğru ayarlayın (örn. `'tr'`, `'en'`, `'de'`).

**Sorun Giderme**
- `404` hatası: `base_url` değerini ve uç noktayı doğrulayın.
- `Timeout`: Büyük dosyalarda `timeout` değerini yükseltin.
- `RateLimit`: `rate_limit_delay` ve `max_retries` değerlerini artırın.
- Desteklenen dilleri görmek için `LanguageHelper.list_languages()` kullanın.
